# Step 10: 대규모 언어 모델(LLM) 이해와 활용

이제 실제 대규모 언어 모델들을 이해하고 활용하는 방법을 배워봅시다.

## 학습 목표
1. 현대 LLM의 발전 과정과 종류
2. Hugging Face Transformers 사용법
3. Fine-tuning과 Prompt Engineering
4. LLM의 한계와 윤리적 고려사항
5. 실제 응용 사례 구현

In [ ]:
# 필요한 라이브러리 설치 (처음 실행 시)
# !pip install transformers datasets accelerate sentencepiece

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    pipeline,
    Trainer,
    TrainingArguments
)
import warnings
warnings.filterwarnings('ignore')

# 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 1. LLM의 발전 역사와 주요 모델들

In [ ]:
def visualize_llm_evolution():
    """LLM 발전 과정 시각화"""
    
    # 주요 모델들의 정보
    models = [
        {'name': 'GPT', 'year': 2018, 'params': 0.117, 'company': 'OpenAI'},
        {'name': 'BERT', 'year': 2018, 'params': 0.340, 'company': 'Google'},
        {'name': 'GPT-2', 'year': 2019, 'params': 1.5, 'company': 'OpenAI'},
        {'name': 'T5', 'year': 2019, 'params': 11, 'company': 'Google'},
        {'name': 'GPT-3', 'year': 2020, 'params': 175, 'company': 'OpenAI'},
        {'name': 'PaLM', 'year': 2022, 'params': 540, 'company': 'Google'},
        {'name': 'LLaMA', 'year': 2023, 'params': 65, 'company': 'Meta'},
        {'name': 'GPT-4', 'year': 2023, 'params': 1000, 'company': 'OpenAI'},  # 추정치
        {'name': 'Claude', 'year': 2023, 'params': 100, 'company': 'Anthropic'},  # 추정치
    ]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. 시간에 따른 모델 크기 변화
    years = [m['year'] for m in models]
    params = [m['params'] for m in models]
    names = [m['name'] for m in models]
    
    ax1.scatter(years, params, s=100, alpha=0.6)
    for i, name in enumerate(names):
        ax1.annotate(name, (years[i], params[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    ax1.set_yscale('log')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Parameters (Billions)')
    ax1.set_title('Evolution of LLM Model Sizes')
    ax1.grid(True, alpha=0.3)
    
    # 2. 회사별 모델 분포
    companies = {}
    for m in models:
        company = m['company']
        if company not in companies:
            companies[company] = []
        companies[company].append(m['name'])
    
    company_names = list(companies.keys())
    model_counts = [len(companies[c]) for c in company_names]
    
    colors = plt.cm.Set3(range(len(company_names)))
    ax2.pie(model_counts, labels=company_names, colors=colors, autopct='%1.1f%%')
    ax2.set_title('LLM Models by Company')
    
    plt.tight_layout()
    plt.show()
    
    # 주요 아키텍처 설명
    print("\n=== 주요 LLM 아키텍처 ===")
    print("\n1. **Encoder-only (BERT 계열)**:")
    print("   - 양방향 문맥 이해")
    print("   - 주로 분류, 추출 작업에 사용")
    print("   - 예: BERT, RoBERTa, ELECTRA")
    
    print("\n2. **Decoder-only (GPT 계열)**:")
    print("   - 자기회귀적 생성")
    print("   - 텍스트 생성에 특화")
    print("   - 예: GPT, GPT-2, GPT-3, LLaMA")
    
    print("\n3. **Encoder-Decoder (T5 계열)**:")
    print("   - 모든 작업을 텍스트 생성으로 통일")
    print("   - 번역, 요약 등에 효과적")
    print("   - 예: T5, BART, mT5")

visualize_llm_evolution()

## 2. Hugging Face Transformers 기초

In [ ]:
# 사전 학습된 모델 불러오기
def load_pretrained_model(model_name="gpt2"):
    """사전 학습된 모델과 토크나이저 로드"""
    print(f"모델 로딩 중: {model_name}")
    
    # 토크나이저와 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    # 모델 정보
    total_params = sum(p.numel() for p in model.parameters())
    print(f"모델 로드 완료!")
    print(f"총 파라미터 수: {total_params:,}")
    print(f"모델 크기: {total_params * 4 / 1024 / 1024:.2f} MB")
    
    return tokenizer, model

# GPT-2 모델 로드
tokenizer, model = load_pretrained_model("gpt2")

# 간단한 텍스트 생성
def generate_text(prompt, model, tokenizer, max_length=50):
    """텍스트 생성 함수"""
    # 입력 인코딩
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.8,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 텍스트 생성 예시
prompt = "Deep learning is"
generated = generate_text(prompt, model, tokenizer)
print(f"\n프롬프트: '{prompt}'")
print(f"생성된 텍스트: '{generated}'")

## 3. Pipeline API 사용하기

In [ ]:
# 다양한 작업을 위한 파이프라인
def demonstrate_pipelines():
    """다양한 NLP 작업 파이프라인 시연"""
    
    print("=== 1. 텍스트 생성 ===")
    generator = pipeline("text-generation", model="gpt2")
    result = generator(
        "The future of AI is",
        max_length=50,
        num_return_sequences=2
    )
    for i, text in enumerate(result):
        print(f"생성 {i+1}: {text['generated_text']}")
    
    print("\n=== 2. 감성 분석 ===")
    sentiment = pipeline("sentiment-analysis")
    texts = [
        "I love this new deep learning framework!",
        "This is terrible and doesn't work at all.",
        "The performance is okay, nothing special."
    ]
    results = sentiment(texts)
    for text, result in zip(texts, results):
        print(f"텍스트: '{text}'")
        print(f"결과: {result['label']} (신뢰도: {result['score']:.3f})\n")
    
    print("=== 3. 질문 답변 ===")
    qa = pipeline("question-answering")
    context = """Deep learning is a subset of machine learning that uses 
    artificial neural networks with multiple layers. It was inspired by 
    the structure and function of the human brain."""
    
    questions = [
        "What is deep learning?",
        "What inspired deep learning?"
    ]
    
    for question in questions:
        result = qa(question=question, context=context)
        print(f"질문: {question}")
        print(f"답변: {result['answer']} (신뢰도: {result['score']:.3f})\n")
    
    print("=== 4. 요약 ===")
    summarizer = pipeline("summarization")
    article = """The Transformer architecture has revolutionized natural language processing. 
    Introduced in the paper 'Attention is All You Need', it replaced recurrent layers with 
    self-attention mechanisms. This allows for better parallelization and capturing of 
    long-range dependencies. The architecture consists of an encoder and decoder, each 
    with multiple layers of multi-head attention and feed-forward networks."""
    
    summary = summarizer(article, max_length=50, min_length=20)
    print(f"원문 길이: {len(article.split())} 단어")
    print(f"요약: {summary[0]['summary_text']}")
    print(f"요약 길이: {len(summary[0]['summary_text'].split())} 단어")

demonstrate_pipelines()

## 4. Prompt Engineering

In [ ]:
class PromptEngineering:
    """프롬프트 엔지니어링 기법들"""
    
    @staticmethod
    def zero_shot(model, tokenizer, task_description, input_text):
        """Zero-shot 프롬프팅"""
        prompt = f"{task_description}\n\nInput: {input_text}\nOutput:"
        return generate_text(prompt, model, tokenizer, max_length=100)
    
    @staticmethod
    def few_shot(model, tokenizer, examples, input_text):
        """Few-shot 프롬프팅"""
        prompt = ""
        for ex in examples:
            prompt += f"Input: {ex['input']}\nOutput: {ex['output']}\n\n"
        prompt += f"Input: {input_text}\nOutput:"
        return generate_text(prompt, model, tokenizer, max_length=100)
    
    @staticmethod
    def chain_of_thought(model, tokenizer, question):
        """Chain-of-thought 프롬프팅"""
        prompt = f"""Question: {question}
Let's think step by step:
1."""
        return generate_text(prompt, model, tokenizer, max_length=150)

# 프롬프트 엔지니어링 예시
pe = PromptEngineering()

print("=== 1. Zero-shot 예시 ===")
result = pe.zero_shot(
    model, tokenizer,
    "Translate English to French:",
    "Hello, how are you?"
)
print(result)

print("\n=== 2. Few-shot 예시 ===")
examples = [
    {"input": "happy", "output": "positive"},
    {"input": "sad", "output": "negative"},
    {"input": "excited", "output": "positive"}
]
result = pe.few_shot(model, tokenizer, examples, "disappointed")
print(result)

print("\n=== 3. Chain-of-thought 예시 ===")
result = pe.chain_of_thought(
    model, tokenizer,
    "If a train travels 60 km/h for 2 hours, how far does it go?"
)
print(result)

## 5. Fine-tuning 기초

In [ ]:
# 간단한 Fine-tuning 예시 (감성 분석)
from datasets import Dataset
import pandas as pd

def prepare_sentiment_dataset():
    """감성 분석 데이터셋 준비"""
    # 예시 데이터
    data = {
        'text': [
            "This movie is fantastic! Best I've seen all year.",
            "Terrible film. Complete waste of time.",
            "Amazing performance by the lead actor.",
            "Boring plot and poor character development.",
            "A masterpiece of modern cinema.",
            "I fell asleep halfway through."
        ],
        'label': [1, 0, 1, 0, 1, 0]  # 1: positive, 0: negative
    }
    
    dataset = Dataset.from_dict(data)
    return dataset

# 데이터셋 토큰화
def tokenize_function(examples, tokenizer):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=128)

# Fine-tuning 설정 시각화
def visualize_finetuning_process():
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Fine-tuning vs Training from scratch
    epochs = np.arange(0, 50)
    from_scratch = 100 * np.exp(-epochs/20) + 10
    fine_tuned = 30 * np.exp(-epochs/5) + 10
    
    axes[0, 0].plot(epochs, from_scratch, 'b-', label='From Scratch', linewidth=2)
    axes[0, 0].plot(epochs, fine_tuned, 'r-', label='Fine-tuned', linewidth=2)
    axes[0, 0].set_xlabel('Epochs')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training Efficiency: Fine-tuning vs From Scratch')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 데이터 요구량
    data_sizes = [10, 100, 1000, 10000, 100000]
    scratch_acc = [0.5, 0.6, 0.75, 0.85, 0.92]
    finetune_acc = [0.7, 0.82, 0.88, 0.91, 0.93]
    
    axes[0, 1].semilogx(data_sizes, scratch_acc, 'b-o', label='From Scratch', linewidth=2)
    axes[0, 1].semilogx(data_sizes, finetune_acc, 'r-o', label='Fine-tuned', linewidth=2)
    axes[0, 1].set_xlabel('Training Data Size')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Data Efficiency')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Fine-tuning 전략
    strategies = ['Full\nFine-tuning', 'Feature\nExtraction', 'LoRA', 'Adapter', 'Prompt\nTuning']
    params_tuned = [100, 0, 0.1, 1, 0.01]  # % of parameters tuned
    performance = [95, 85, 92, 90, 88]
    
    x = np.arange(len(strategies))
    width = 0.35
    
    ax1 = axes[1, 0]
    ax2 = ax1.twinx()
    
    bars1 = ax1.bar(x - width/2, params_tuned, width, label='Parameters Tuned (%)', color='skyblue')
    bars2 = ax2.bar(x + width/2, performance, width, label='Performance (%)', color='lightcoral')
    
    ax1.set_xlabel('Fine-tuning Strategy')
    ax1.set_ylabel('Parameters Tuned (%)', color='skyblue')
    ax2.set_ylabel('Performance (%)', color='lightcoral')
    ax1.set_title('Fine-tuning Strategies Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels(strategies)
    ax1.tick_params(axis='y', labelcolor='skyblue')
    ax2.tick_params(axis='y', labelcolor='lightcoral')
    
    # 4. 학습률 스케줄
    steps = np.arange(0, 1000)
    warmup_steps = 100
    
    # Linear warmup + cosine decay
    lr = np.ones_like(steps, dtype=float)
    lr[:warmup_steps] = steps[:warmup_steps] / warmup_steps
    lr[warmup_steps:] = 0.5 * (1 + np.cos(np.pi * (steps[warmup_steps:] - warmup_steps) / (1000 - warmup_steps)))
    
    axes[1, 1].plot(steps, lr, 'g-', linewidth=2)
    axes[1, 1].axvline(x=warmup_steps, color='r', linestyle='--', alpha=0.5, label='End of Warmup')
    axes[1, 1].set_xlabel('Training Steps')
    axes[1, 1].set_ylabel('Learning Rate (Relative)')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_finetuning_process()

print("\n=== Fine-tuning 팁 ===")
print("1. **학습률**: 사전학습보다 낮은 학습률 사용 (1e-5 ~ 1e-4)")
print("2. **Warmup**: 초기 학습률을 점진적으로 증가")
print("3. **레이어 동결**: 하위 레이어는 동결하고 상위 레이어만 학습")
print("4. **정규화**: Dropout, weight decay 활용")
print("5. **조기 종료**: Validation loss 모니터링")

## 6. 효율적인 LLM 사용법

In [ ]:
class EfficientLLM:
    """효율적인 LLM 사용 기법"""
    
    @staticmethod
    def quantization_demo():
        """양자화로 모델 크기 줄이기"""
        print("=== 양자화 (Quantization) ===")
        print("원리: 가중치를 낮은 비트로 표현")
        print("- FP32 (32-bit) → INT8 (8-bit): 75% 메모리 절약")
        print("- FP32 → INT4 (4-bit): 87.5% 메모리 절약")
        print("- 성능 손실: 일반적으로 1-3%")
        
        # 시각화
        bit_widths = [32, 16, 8, 4]
        memory_usage = [100, 50, 25, 12.5]
        accuracy = [100, 99.5, 98, 95]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        ax1.bar(bit_widths, memory_usage, color='skyblue')
        ax1.set_xlabel('Bit Width')
        ax1.set_ylabel('Memory Usage (%)')
        ax1.set_title('Memory Usage vs Bit Width')
        ax1.set_xticks(bit_widths)
        
        ax2.plot(bit_widths, accuracy, 'ro-', linewidth=2, markersize=8)
        ax2.set_xlabel('Bit Width')
        ax2.set_ylabel('Accuracy (%)')
        ax2.set_title('Accuracy vs Bit Width')
        ax2.set_xticks(bit_widths)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def efficient_attention():
        """효율적인 Attention 메커니즘"""
        print("\n=== 효율적인 Attention ===")
        
        methods = {
            'Standard Attention': 'O(n²) 메모리, 모든 토큰 쌍 계산',
            'Flash Attention': '메모리 효율적, IO-aware 알고리즘',
            'Sparse Attention': '일부 토큰만 attend, O(n√n)',
            'Linear Attention': 'Kernel trick 사용, O(n)',
            'Local Attention': '윈도우 내에서만 attention'
        }
        
        for method, description in methods.items():
            print(f"- {method}: {description}")
    
    @staticmethod
    def model_parallelism():
        """모델 병렬화 전략"""
        print("\n=== 모델 병렬화 ===")
        print("1. **Data Parallelism**: 데이터를 나누어 처리")
        print("2. **Model Parallelism**: 모델을 나누어 처리")
        print("3. **Pipeline Parallelism**: 레이어를 파이프라인으로")
        print("4. **Tensor Parallelism**: 텐서 연산을 분할")

# 효율적인 LLM 사용법 시연
efficient = EfficientLLM()
efficient.quantization_demo()
efficient.efficient_attention()
efficient.model_parallelism()

## 7. LLM의 한계와 윤리적 고려사항

In [ ]:
def discuss_llm_limitations():
    """LLM의 한계와 윤리적 이슈"""
    
    # 한계점 시각화
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Hallucination 비율
    tasks = ['Facts', 'Math', 'Code', 'Creative', 'Common Sense']
    hallucination_rates = [15, 25, 20, 5, 30]
    
    axes[0, 0].bar(tasks, hallucination_rates, color='coral')
    axes[0, 0].set_ylabel('Hallucination Rate (%)')
    axes[0, 0].set_title('Hallucination Rates by Task Type')
    axes[0, 0].set_ylim(0, 40)
    
    # 2. 컨텍스트 길이 제한
    models = ['GPT-2', 'GPT-3', 'GPT-4', 'Claude', 'Gemini']
    context_lengths = [1024, 4096, 8192, 100000, 128000]
    
    axes[0, 1].bar(models, np.log10(context_lengths), color='skyblue')
    axes[0, 1].set_ylabel('Log10(Context Length)')
    axes[0, 1].set_title('Context Length Limitations')
    
    # Y축 라벨 수정
    y_labels = [f'{10**i:,}' for i in range(3, 6)]
    axes[0, 1].set_yticks(range(3, 6))
    axes[0, 1].set_yticklabels(y_labels)
    
    # 3. 편향성 예시
    bias_types = ['Gender', 'Race', 'Religion', 'Politics', 'Culture']
    bias_scores = [0.7, 0.8, 0.6, 0.9, 0.75]
    
    axes[1, 0].barh(bias_types, bias_scores, color='lightgreen')
    axes[1, 0].set_xlabel('Bias Score (0-1)')
    axes[1, 0].set_title('Types of Bias in LLMs')
    axes[1, 0].set_xlim(0, 1)
    
    # 4. 환경 영향
    model_sizes = ['1B', '10B', '100B', '1T']
    co2_emissions = [0.1, 10, 1000, 100000]  # kg CO2
    
    axes[1, 1].semilogy(model_sizes, co2_emissions, 'ro-', linewidth=2, markersize=8)
    axes[1, 1].set_xlabel('Model Size (Parameters)')
    axes[1, 1].set_ylabel('CO2 Emissions (kg)')
    axes[1, 1].set_title('Environmental Impact of Training LLMs')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== LLM의 주요 한계 ===")
    print("\n1. **Hallucination (환각)**:")
    print("   - 그럴듯하지만 잘못된 정보 생성")
    print("   - 출처나 근거 없는 주장")
    print("   - 해결책: 사실 확인, 출처 요구")
    
    print("\n2. **컨텍스트 제한**:")
    print("   - 긴 문서 처리의 한계")
    print("   - 대화 기록 손실")
    print("   - 해결책: 요약, 청킹, 검색 증강")
    
    print("\n3. **편향성**:")
    print("   - 학습 데이터의 편향 반영")
    print("   - 사회적 스테레오타입 강화")
    print("   - 해결책: 다양한 데이터, 편향 감지")
    
    print("\n4. **프라이버시**:")
    print("   - 학습 데이터 유출 가능성")
    print("   - 개인정보 노출 위험")
    print("   - 해결책: 차등 프라이버시, 데이터 필터링")
    
    print("\n5. **환경 영향**:")
    print("   - 막대한 전력 소비")
    print("   - 탄소 배출")
    print("   - 해결책: 효율적 아키텍처, 재생 에너지")

discuss_llm_limitations()

## 8. 실제 응용 사례 구현

In [ ]:
class LLMApplications:
    """LLM을 활용한 실제 응용 사례"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def code_assistant(self, description):
        """코드 생성 도우미"""
        prompt = f"""# Python function to {description}
def"""
        generated = generate_text(prompt, self.model, self.tokenizer, max_length=150)
        return generated
    
    def creative_writing(self, genre, theme):
        """창의적 글쓰기"""
        prompt = f"""Write a short {genre} story about {theme}:

Once upon a time,"""
        generated = generate_text(prompt, self.model, self.tokenizer, max_length=200)
        return generated
    
    def data_analysis_helper(self, data_description):
        """데이터 분석 도우미"""
        prompt = f"""Given the following data: {data_description}
        
Analysis steps:
1. First, we should"""
        generated = generate_text(prompt, self.model, self.tokenizer, max_length=150)
        return generated
    
    def educational_tutor(self, subject, question):
        """교육 도우미"""
        prompt = f"""As a {subject} tutor, explain: {question}
        
Explanation:"""
        generated = generate_text(prompt, self.model, self.tokenizer, max_length=200)
        return generated

# 응용 사례 실행
apps = LLMApplications(model, tokenizer)

print("=== 1. 코드 생성 도우미 ===")
code = apps.code_assistant("calculate fibonacci numbers")
print(code)

print("\n=== 2. 창의적 글쓰기 ===")
story = apps.creative_writing("science fiction", "time travel")
print(story[:300] + "...")

print("\n=== 3. 데이터 분석 도우미 ===")
analysis = apps.data_analysis_helper("customer purchase data with timestamps and amounts")
print(analysis)

print("\n=== 4. 교육 도우미 ===")
explanation = apps.educational_tutor("physics", "how does gravity work")
print(explanation[:300] + "...")

## 9. LLM의 미래

In [ ]:
def visualize_llm_future():
    """LLM의 미래 전망 시각화"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. 모델 크기 예측
    years = np.array([2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025])
    model_sizes = np.array([0.1, 1.5, 175, 280, 540, 1000, 2000, 5000])  # Billions
    
    axes[0, 0].semilogy(years[:6], model_sizes[:6], 'bo-', label='Historical', linewidth=2)
    axes[0, 0].semilogy(years[5:], model_sizes[5:], 'r--', label='Projected', linewidth=2)
    axes[0, 0].set_xlabel('Year')
    axes[0, 0].set_ylabel('Model Size (Billion Parameters)')
    axes[0, 0].set_title('Growth of LLM Sizes')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 멀티모달 능력
    modalities = ['Text', 'Image', 'Audio', 'Video', '3D']
    current = [100, 80, 60, 30, 10]
    future = [100, 95, 90, 80, 60]
    
    x = np.arange(len(modalities))
    width = 0.35
    
    axes[0, 1].bar(x - width/2, current, width, label='Current', color='lightblue')
    axes[0, 1].bar(x + width/2, future, width, label='Future (2025)', color='lightgreen')
    axes[0, 1].set_ylabel('Capability Level (%)')
    axes[0, 1].set_title('Multimodal Capabilities')
    axes[0, 1].set_xticks(x)
    axes[0, 1].set_xticklabels(modalities)
    axes[0, 1].legend()
    
    # 3. 응용 분야
    applications = [
        'Healthcare', 'Education', 'Research', 
        'Entertainment', 'Business', 'Personal Assistant'
    ]
    adoption_rates = [40, 50, 70, 60, 80, 90]
    
    axes[1, 0].barh(applications, adoption_rates, color='coral')
    axes[1, 0].set_xlabel('Adoption Rate (%)')
    axes[1, 0].set_title('LLM Applications by 2025')
    axes[1, 0].set_xlim(0, 100)
    
    # 4. 기술 발전 로드맵
    timeline = ['2024', '2025', '2026', '2027', '2028']
    milestones = [
        'Efficient\nArchitectures',
        'Real-time\nMultimodal',
        'Personal\nAI Agents',
        'Scientific\nDiscovery',
        'AGI\nCapabilities?'
    ]
    
    axes[1, 1].scatter(timeline, range(len(timeline)), s=200, c='gold', edgecolors='black', linewidth=2)
    for i, (time, milestone) in enumerate(zip(timeline, milestones)):
        axes[1, 1].text(i, i + 0.1, milestone, ha='center', fontsize=10)
    
    axes[1, 1].set_xlabel('Year')
    axes[1, 1].set_title('LLM Development Roadmap')
    axes[1, 1].set_ylim(-0.5, len(timeline) - 0.5)
    axes[1, 1].set_yticks([])
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== LLM의 미래 전망 ===")
    print("\n1. **더 효율적인 모델**:")
    print("   - Sparse models")
    print("   - Mixture of Experts")
    print("   - Neural Architecture Search")
    
    print("\n2. **멀티모달 통합**:")
    print("   - 텍스트 + 이미지 + 오디오 + 비디오")
    print("   - 실시간 처리")
    print("   - 3D 이해")
    
    print("\n3. **개인화된 AI**:")
    print("   - 개인 맞춤형 모델")
    print("   - 지속적 학습")
    print("   - 프라이버시 보장")
    
    print("\n4. **과학적 발견**:")
    print("   - 신약 개발")
    print("   - 재료 과학")
    print("   - 기후 모델링")
    
    print("\n5. **윤리적 AI**:")
    print("   - 설명 가능한 AI")
    print("   - 편향 제거")
    print("   - 안전한 AI")

visualize_llm_future()

## 10. 실습: 나만의 LLM 응용 프로그램

In [ ]:
# 간단한 챗봇 구현
class SimpleChatbot:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.conversation_history = []
    
    def chat(self, user_input):
        # 대화 기록에 추가
        self.conversation_history.append(f"User: {user_input}")
        
        # 컨텍스트 생성 (최근 5개 대화만 유지)
        context = "\n".join(self.conversation_history[-5:])
        prompt = f"{context}\nAssistant:"
        
        # 응답 생성
        response = generate_text(prompt, self.model, self.tokenizer, max_length=100)
        
        # 응답 추출
        response_text = response.split("Assistant:")[-1].strip()
        if "User:" in response_text:
            response_text = response_text.split("User:")[0].strip()
        
        self.conversation_history.append(f"Assistant: {response_text}")
        
        return response_text
    
    def reset(self):
        self.conversation_history = []

# 챗봇 테스트
chatbot = SimpleChatbot(model, tokenizer)

print("=== 간단한 챗봇 데모 ===")
print("(입력 'quit'으로 종료)\n")

# 미리 정의된 대화 예시
test_conversations = [
    "Hello, how are you?",
    "What is machine learning?",
    "Can you explain it simply?"
]

for user_input in test_conversations:
    print(f"You: {user_input}")
    response = chatbot.chat(user_input)
    print(f"Bot: {response}\n")

print("\n=== 대화 기록 ===")
for line in chatbot.conversation_history:
    print(line)

## 연습 문제

In [ ]:
# 문제 1: RAG (Retrieval-Augmented Generation) 구현
class SimpleRAG:
    """
    검색 증강 생성 구현
    문서에서 관련 정보를 검색하고 LLM으로 답변 생성
    """
    def __init__(self, documents, model, tokenizer):
        self.documents = documents
        self.model = model
        self.tokenizer = tokenizer
    
    def retrieve(self, query, k=3):
        # TODO: 쿼리와 가장 관련 있는 k개 문서 검색
        # 힌트: 간단한 TF-IDF 또는 임베딩 유사도 사용
        pass
    
    def generate_answer(self, query, retrieved_docs):
        # TODO: 검색된 문서를 바탕으로 답변 생성
        pass

# 문제 2: 프롬프트 최적화
def optimize_prompt(task, examples, constraints):
    """
    주어진 작업에 대한 최적의 프롬프트 생성
    
    힌트:
    - 명확한 지시사항
    - 예시 포함
    - 제약사항 명시
    """
    # TODO: 구현하기
    pass

# 문제 3: 모델 평가 메트릭
def evaluate_llm_output(generated_text, reference_text):
    """
    LLM 출력 품질 평가
    
    평가 기준:
    - 유창성 (Fluency)
    - 일관성 (Coherence)
    - 관련성 (Relevance)
    - 정확성 (Accuracy)
    """
    # TODO: 구현하기
    pass

## 정리

이번 튜토리얼에서 배운 내용:
1. **LLM의 발전**: GPT, BERT부터 최신 모델까지
2. **Hugging Face 활용**: 사전학습 모델 사용법
3. **Prompt Engineering**: 효과적인 프롬프트 작성
4. **Fine-tuning**: 특정 작업에 맞게 조정
5. **효율적 사용**: 양자화, 효율적 attention
6. **한계와 윤리**: Hallucination, 편향성, 환경 영향
7. **실제 응용**: 코드 생성, 창작, 교육 등

### LLM 사용 시 기억할 점:
- **검증**: 생성된 내용 항상 확인
- **윤리**: 편향성과 해로운 콘텐츠 주의
- **효율성**: 적절한 모델 크기 선택
- **프라이버시**: 민감한 데이터 주의

### 다음 학습 추천:
1. **고급 기법**: LoRA, QLoRA, PEFT
2. **멀티모달**: CLIP, DALL-E, Flamingo
3. **특화 모델**: CodeLLM, BioLLM, LegalLLM
4. **배포**: Model serving, API 구축

축하합니다! 이제 여러분은 딥러닝의 기초부터 최신 LLM까지 전체적인 이해를 갖추었습니다! 🎉